In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, RationalQuadratic
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import math
import optuna
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import torch
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import torch.multiprocessing as mp
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
from rdkit.Chem import rdMolDescriptors
import os
from sklearn.inspection import permutation_importance
import shap

In [ ]:
df=pd.read_csv('ART_main.csv',encoding='ISO-8859-1')
df

In [ ]:
outputs=np.array(df['ee'])
mols=[Chem.MolFromSmiles(i) for i in df['smiles']]
features=[AllChem.GetMorganFingerprintAsBitVect(i, 2, nBits=1024) for i in mols]
features1=[rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=1024) for mol in mols]
features2=[Chem.LayeredFingerprint(mol, fpSize=1024) for mol in mols]

fps1= np.array(features)
fps2= np.array(features1)
fps3= np.array(features2)

concatenated_fps = np.concatenate((fps1, fps2, fps3), axis=1)
concatenated_fps_df = pd.DataFrame(concatenated_fps)
concatenated_fps_df['ee'] = outputs
print(concatenated_fps_df.shape)

In [ ]:
X = np.array(concatenated_fps_df.iloc[:,:-1])
y=np.array(concatenated_fps_df['ee']).reshape(-1, 1)

In [ ]:
# Function to split the dataset and preprocess it
def dataset_func(X, y, rand_state):
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.12, random_state=rand_state)

    print("Train Dataset: {}".format(X_train.shape))
    print("Val Dataset: {}".format(X_val.shape))
    print("Test Dataset: {}".format(X_test.shape))

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
# Function to split the dataset and preprocess i

# Function to define and train the Random Forest Regressor model
def train_random_forest(X_train, y_train, n_estimators, max_depth, rand_state):
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=rand_state)
    model.fit(X_train, y_train)
    return model

# Function to define and train the Random Forest Regressor model
def train_decision_tree(X_train, y_train, max_depth, rand_state):
    model = DecisionTreeRegressor(max_depth=max_depth, random_state=rand_state)
    model.fit(X_train, y_train)
    return model


# Function to define and train the Gradient Boosting Regressor model
def train_gradient_boosting(X_train, y_train, n_estimators, max_depth, rand_state):
    model = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=rand_state)
    model.fit(X_train, y_train)
    return model

class SVMPredictor:
    def __init__(self, svm_callable):
        self.svm_callable = svm_callable

    def predict(self, X):
        return self.svm_callable(X)

# Modify train_svm to return an instance of SVMPredictor
def train_svm(X_train, y_train, C, epsilon, kernel):
    model = SVR(C=C, epsilon=epsilon, kernel=kernel)
    model.fit(X_train, y_train)

    # Return an instance of SVMPredictor
    return SVMPredictor(lambda x: model.predict(x))


# Function to evaluate the model on train, validation, and test sets
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test):
    train_predictions = model.predict(X_train)
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)

    train_rmse = math.sqrt(mean_squared_error(y_train, train_predictions))
    val_rmse = math.sqrt(mean_squared_error(y_val, val_predictions))
    test_rmse = math.sqrt(mean_squared_error(y_test, test_predictions))

    train_r2 = r2_score(y_train, train_predictions)
    val_r2 = r2_score(y_val, val_predictions)
    test_r2 = r2_score(y_test, test_predictions)

    return train_rmse, val_rmse, test_rmse, train_r2, val_r2, test_r2

def plot_permutation_importance(model, X, y, save_path):
    if isinstance(X, pd.DataFrame):
        feature_names = X.columns
    elif isinstance(X, np.ndarray):
        feature_names = np.arange(X.shape[1])
    else:
        raise ValueError("Unsupported input type. Use either Pandas DataFrame or NumPy array.")

    result = permutation_importance(model, X, y, n_repeats=30, random_state=0)
    sorted_idx = result.importances_mean.argsort()
    

    plt.barh(range(20), result.importances_mean[sorted_idx[-20:]])
    plt.yticks(range(20), feature_names[sorted_idx[0:20]])
    plt.xlabel('Permutation Importance')
    plt.title('Permutation Feature Importance')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

def plot_shap_importance(model, X, save_path):
    explainer = shap.Explainer(model)
    shap_values = explainer.shap_values(X)

    shap.summary_plot(shap_values, X, show=False)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

def plot_shap_importance_svm_gpr(model, X_tr, X_val, save_path):
    # Create a Kernel SHAP explainer
    explainer = shap.Explainer(model.predict, X_tr)
    
    # Compute SHAP values for the test set
    shap_values = explainer.shap_values(X_val)
    
    shap.summary_plot(shap_values, X_val, show=False)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

def objective_rf(trial, X_train, y_train, X_val, y_val):
    n_estimators = trial.suggest_int('n_estimators', 1,100)
    max_depth = trial.suggest_int('max_depth', 1, 100)
    model = train_random_forest(X_train, y_train, n_estimators, max_depth, rand_state)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(y_val, val_preds)
    train_preds = model.predict(X_train)
    train_loss = mean_squared_error(y_train, train_preds)
    obj_val = val_loss-train_loss
    #print('train_loss:',train_loss)
    #print('val_loss:',val_loss)
    return val_loss


def objective_dt(trial, X_train, y_train, X_val, y_val):
    max_depth = trial.suggest_int('max_depth', 1, 100)
    model = train_decision_tree(X_train, y_train, max_depth, rand_state)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(y_val, val_preds)
    return val_loss
    

def objective_gb(trial, X_train, y_train, X_val, y_val):
    n_estimators = trial.suggest_int('n_estimators', 1, 100)
    max_depth = trial.suggest_int('max_depth', 1, 100)
    model = train_gradient_boosting(X_train, y_train, n_estimators, max_depth, rand_state)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(y_val, val_preds)
    return val_loss

def objective_svm(trial, X_train, y_train, X_val, y_val):
    C = trial.suggest_loguniform('C', 1, 30)
    epsilon = trial.suggest_loguniform('epsilon', 1e-5, 1e5)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    model = train_svm(X_train, y_train, C, epsilon, kernel)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(y_val, val_preds)
    return val_loss


def objective_func_for_model(model_name,X_train, y_train, X_val, y_val,trial):
    if model_name == 'Random Forest':
        return objective_rf(trial, X_train, y_train, X_val, y_val)
    elif model_name == 'Decision tree':
        return objective_dt(trial, X_train, y_train, X_val, y_val)
    elif model_name == 'Gradient Boosting':
        return objective_gb(trial, X_train, y_train, X_val, y_val)
    elif model_name == 'SVM':
        return objective_svm(trial, X_train, y_train, X_val, y_val)
    else:
        raise ValueError(f"Unknown model: {model_name}")

def optimize_model(params,save_path_permutation, save_path_shap):
    seed, model_name, objective_func, n_trials, X, y = params
    study = optuna.create_study(direction='minimize')
    X_train, y_train, X_val, y_val, X_test, y_test = dataset_func(X, y, seed)
    
    # Modify the partial function to pass additional arguments
    objective_func_partial = partial(objective_func_for_model, model_name, X_train, y_train, X_val, y_val)
    
    study.optimize(objective_func_partial, n_trials)
    best_params = study.best_params
    final_model = None

    if model_name == 'Random Forest':
        final_model = train_random_forest(X_train, y_train, best_params['n_estimators'], best_params['max_depth'], seed)
        #plot_permutation_importance(final_model, X_val, y_val, save_path_permutation)
        #plot_shap_importance(final_model, X_val, save_path_shap)
    elif model_name == 'Decision tree':
        final_model = train_decision_tree(X_train, y_train, best_params['max_depth'], seed)
        #plot_permutation_importance(final_model, X_val, y_val, save_path_permutation)
        #plot_shap_importance(final_model, X_val, save_path_shap)
    elif model_name == 'Gradient Boosting':
        final_model = train_gradient_boosting(X_train, y_train, best_params['n_estimators'], best_params['max_depth'], seed)
        #plot_permutation_importance(final_model, X_val, y_val, save_path_permutation)
        #plot_shap_importance(final_model, X_val, save_path_shap)
    elif model_name == 'SVM':
        final_model = train_svm(X_train, y_train, best_params['C'], best_params['epsilon'], best_params['kernel'])
        #plot_shap_importance_svm_gpr(final_model, X_train, X_val, save_path_shap)

    train_rmse, val_rmse, test_rmse, train_r2, val_r2, test_r2 = evaluate_model(final_model, X_train, y_train, X_val, y_val, X_test, y_test)

    return {'Model': model_name, 'Seed': seed,
            'Train RMSE': train_rmse, 'Val RMSE': val_rmse, 'Test RMSE': test_rmse,
            'Train R2': train_r2, 'Val R2': val_r2, 'Test R2': test_r2, 'Best Parameters': best_params}


In [ ]:
# Set random state
rand_states = [i for i in range(0,30)]  # Add more seed values as needed

# Create an empty list to store futures
futures = []

# Use a ProcessPoolExecutor for parallel processing
with ProcessPoolExecutor() as executor:
    for model_name in ['Random Forest','Decision tree', 'Gradient Boosting','SVM',]: # , 'Random Forest','GPR' , 'Gradient Boosting','SVM'
        for rand_state in rand_states:
            # Pass parameters as a tuple
            params = (rand_state, model_name, objective_func_for_model, 100, X, y)
            save_path_permutation = f'{model_name}_seed_{rand_state}_permutation_importance_plot.png'
            save_path_shap = f'{model_name}_seed_{rand_state}_shap_importance_plot.png'
            # Submit each optimization task as a future
            futures.append(executor.submit(optimize_model, params, save_path_permutation, save_path_shap))

# Collect the results from completed futures
results_list = [future.result() for future in futures]

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results_list)


In [ ]:
# Assuming df is your DataFrame
results_df_rounded = results_df.round(3)
results_df_rounded

In [ ]:
results_df_rounded.to_csv('performance_file.csv')